# [Module 4] Distributed training with horovod

Horovod는 MPI(Message Passing Interface; 메세지 전달 인터페이스)를 기반으로 하는 분산 학습 프레임워크(distributed training framework)입니다. Horovod는 TensorFlow 버전 1.12 이상에서만 사용할 수 있습니다. 자세한 내용은 [Horovod README](https://github.com/uber/horovod)에서 확인할 수 있습니다.

Horovod를 활성화하려면 학습 스크립트를 약간 수정해야 합니다. 본 실습에서 이를 직접 수행해 보겠습니다.

## Create a training script that support Horovod distributed training

`training_script/cifar10_keras_sm.py`의 사본을 생성 후 **<font color='red'>(주의: `training_script/cifar10_keras_pipe.py`의 사본이 아닙니다)</font>**, `training_script/cifar10_keras_dist.py` 로 저장하세요.

스크립트 사본을 생성하였다면 단계별로 아래의 작업들을 직접 시도합니다.

----
### TODO 1. Start horovod
Horovod에 대응하기 위해 `main()` 함수에 아래 코드를 추가합니다.

```python
    import horovod.keras as hvd
    hvd.init()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.visible_device_list = str(hvd.local_rank())
    K.set_session(tf.Session(config=config))
```

----
### TODO 2. Configure callbacks
`main()` 함수에서 callbacks을 추가합니다.

```python
    callbacks.append(hvd.callbacks.BroadcastGlobalVariablesCallback(0))
    callbacks.append(hvd.callbacks.MetricAverageCallback())
    callbacks.append(hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1))
```

`hvd.rank () == 0` 에서만 실행되도록 체크포인트 및 TensorBoard 콜백을 변경해 주세요.
```python
    if hvd.rank() == 0:
        callbacks.append(ModelCheckpoint(args.output_dir + '/checkpoint-{epoch}.h5'))
        callbacks.append(TensorBoard(log_dir=args.model_output_dir,update_freq='epoch'))
```

----
### TODO 3. Configure the optimizer
Horovod에 대응하기 위해 아래의 절차들을 진행합니다.


1) `keras_model_fn` 함수에 hvd 인수를 추가합니다.
```python
# Add hvd to the function. also add it in the function call
def keras_model_fn(learning_rate, weight_decay, optimizer, momentum, hvd): 
```

2) `size=1`을 `size=hvd.size()`로 변경해 주세요.

3) 코드를 아래와 같이 수정합니다.

```python
 model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
```
바로 앞에
```python
opt = hvd.DistributedOptimizer(opt)
```
라인을 추가해 주세요.

4) `main()` 함수에서 model 인스턴스를 만들 때 hvd를 인수로 전달하도록 수정합니다.

```python
model = keras_model_fn(args.learning_rate, args.weight_decay, args.optimizer, args.momentum, hvd)
```

<font color='blue'>**본 노트북 실습에 어려움이 있다면 솔루션 파일 `training_script/cifar10_keras_dist_solution.py`을 참조하시면 됩니다.**</font>

## Run Distributed training
아래의 설정을 Estimator 객체에 전달하여 Horovod 분산 학습에 대한 설정을 할 수 있습니다.
```python
distributions = {'mpi': {
                    'enabled': True,
                    'processes_per_host': # Number of Horovod processes per host
                        }
                }
```

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

In [2]:
prefix = 'data/DEMO-cifar10'
dataset_location = os.path.join('s3://', sagemaker_session.default_bucket(), prefix)

In [3]:
metric_definitions = [
    {'Name': 'train:loss', 'Regex': 'loss: (.*?) '},
    {'Name': 'train:accuracy', 'Regex': 'acc: (.*?) '},
    {'Name': 'validation:loss', 'Regex': 'val_loss: (.*?) '},
    {'Name': 'validation:accuracy', 'Regex': 'val_acc: (.*?) '}
]

`train_instance_count` 인자값을 2로 설정하고 `distribution` 인자값을 추가합니다.<br>
이번에는 5 epoch 대신 10 epoch를 학습합니다.

In [4]:
from sagemaker.tensorflow import TensorFlow

distributions = {
    'mpi': {
        'enabled': True, 
        'custom_mpi_options': '-verbose --NCCL_DEBUG=INFO',
        'processes_per_host': 1
    }
}

# Change base_job_name to 'cifar10-dist' for console visibility
estimator = TensorFlow(base_job_name='cifar10-dist',
                       entry_point='cifar10_keras_dist.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='1.14.0',
                       py_version='py3',
                       script_mode=True,                            
                       hyperparameters={'epochs': 10},
                       train_instance_count=2,   # 변경
                       train_instance_type='ml.p2.xlarge',
                       metric_definitions=metric_definitions, # 1_Monitoring_your_TensorFlow_scripts.ipynb 참조                         
                       distributions=distributions # 추가
                      )

학습 완료 후 Billable seconds도 확인해 보세요. Billable seconds는 실제로 학습 수행 시 과금되는 시간입니다.
```
Billable seconds: <time>
```

참고로, `ml.p2.xlarge` 인스턴스로 10 epoch 학습 시 전체 6분~7분이 소요되고, 실제 학습에 소요되는 시간은 3분~4분이 소요됩니다.

In [5]:
%%time
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

2019-10-21 07:59:04 Starting - Starting the training job...
2019-10-21 07:59:07 Starting - Launching requested ML instances......
2019-10-21 08:00:07 Starting - Preparing the instances for training......
2019-10-21 08:01:15 Downloading - Downloading input data...
2019-10-21 08:01:58 Training - Downloading the training image...
2019-10-21 08:02:30 Training - Training image download completed. Training in progress../usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1

[1,1]<stderr>:I1021 08:02:42.275452 140285834823424 cifar10_keras_dist.py:220] configuring model
[1,0]<stderr>:I1021 08:02:42.289049 139980435289856 cifar10_keras_dist.py:220] configuring model
[ip-10-2-159-196.ec2.internal:00036] 1 more process has sent help message help-orte-odls-default.txt / memory not bound
[ip-10-2-159-196.ec2.internal:00036] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages
[1,0]<stderr>:W1021 08:02:43.943424 139980435289856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.
[1,0]<stderr>:
[1,0]<stderr>:W1021 08:02:44.120131 139980435289856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.
[1,0]<stderr>:
[1,0]<stderr>:W1021 08:02:44

[1,1]<stdout>:#015[1,1]<stdout>: 1/78 [..............................] - ETA: 11:43 - loss: 3.8809 - acc: 0.1094[1,0]<stdout>:#015[1,0]<stdout>: 1/78 [..............................] - ETA: 11:42 - loss: 3.8795 - acc: 0.0898[1,1]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015[1,1]<stdout>: 2/78 [..............................] - ETA: 5:51 - loss: 3.7120 - acc: 0.0918 [1,0]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015[1,0]<

[1,1]<stdout>:#015[1,1]<stdout>: 1/78 [..............................] - ETA: 17s - loss: 1.6840 - acc: 0.3867[1,0]<stdout>:#015[1,0]<stdout>: 1/78 [..............................] - ETA: 8s - loss: 1.7831 - acc: 0.3242[1,0]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015[1,0]<stdout>: 2/78 [..............................] - ETA: 9s - loss: 1.7266 - acc: 0.3340[1,1]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015[1,1]<stdout>: 2/78 [.............

[1,1]<stdout>:#015[1,1]<stdout>: 1/78 [..............................] - ETA: 28s - loss: 1.5223 - acc: 0.4375[1,0]<stdout>:#015[1,0]<stdout>: 1/78 [..............................] - ETA: 9s - loss: 1.4336 - acc: 0.4688[1,1]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015[1,1]<stdout>: 2/78 [..............................] - ETA: 19s - loss: 1.5409 - acc: 0.4258[1,0]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/78 [..........................

[1,1]<stdout>:#015[1,1]<stdout>: 1/78 [..............................] - ETA: 18s - loss: 1.2377 - acc: 0.5547[1,0]<stdout>:#015[1,0]<stdout>: 1/78 [..............................] - ETA: 9s - loss: 1.3874 - acc: 0.5234[1,1]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015[1,1]<stdout>: 2/78 [..............................] - ETA: 14s - loss: 1.2544 - acc: 0.5605[1,0]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015[1,0]<stdout>: 2/78 [............

[1,1]<stdout>:#015[1,1]<stdout>: 1/78 [..............................] - ETA: 17s - loss: 1.2349 - acc: 0.5938[1,0]<stdout>:#015[1,0]<stdout>: 1/78 [..............................] - ETA: 9s - loss: 1.3091 - acc: 0.4883[1,0]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015[1,0]<stdout>: 2/78 [..............................] - ETA: 9s - loss: 1.2642 - acc: 0.5430[1,1]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015[1,1]<stdout>: 2/78 [.............

#010#010#010#010#010#010#010#010#010#010#010#010#015[1,1]<stdout>:46/78 [================>.............] - ETA: 4s - loss: 1.2271 - acc: 0.5592[1,0]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015[1,0]<stdout>:46/78 [================>.............] - ETA: 4s - loss: 1.2230 - acc: 0.5606[1,0]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015[1,0]<stdout>:47/78 [=================>............] - ETA: 4s - loss: 1.2213 - acc: 0.5609[1,1]<stdout>:#010#010#


2019-10-21 08:04:56 Uploading - Uploading generated training model
2019-10-21 08:04:56 Completed - Training job completed
Training seconds: 442
Billable seconds: 442
CPU times: user 926 ms, sys: 71.1 ms, total: 997 ms
Wall time: 6min 14s


**잘 하셨습니다.**  

여러 분은 이제 분산 학습에 SageMaker 학습 작업을 사용할 수 있습니다.
다음 노트북으로 계속 진행하기 전에 CloudWatch 및 TensorBoard의 distribution job metrics를 살펴 보세요.
TensorBoard를 사용하여 여러분이 실행한 다른 작업을 비교할 수 있습니다.

TensorBoard 실행 시 아래의 인자값을 참조해 주세요.<br>
`--logdir dist:dist_model_dir,pipe:pipe_model_dir,file:normal_job_model_dir`